# Inference with pipeline

In [1]:
from transformers import pipeline

/mnt/custom-file-systems/efs/fs-0252e317d4af1dc34_fsap-0a708b50be80889d5/CLONED_REPOS/LLM-World/llmenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

pipe = pipeline(
    'text-generation', # Tells the pipeline we would like to generate text
    model='openai/gpt-oss-20b', # specifies the model
    torch_dtype='auto', # data type the model uses in PyTorch
    device_map='auto') # maps the model to a device

messages = [{'role':'user', 'content':'Explain what mixture-of-experts is'}]

out = pipe(messages,
 max_new_tokens=3000, # number of tokens the model will generate (output length)
 temperature=1.0) # 1 = baseline randomness in token selection

Loading checkpoint shards: 100%|██████████| 3/3 [01:06<00:00, 22.33s/it]
Device set to use cuda:0


In [3]:
out

[{'generated_text': [{'role': 'user',
    'content': 'Explain what mixture-of-experts is'},
   {'role': 'assistant',
    'content': 'analysisThe user asks: "Explain what mixture-of-experts is". Need to provide a clear explanation of the concept: mixture of experts (MoE) architecture in machine learning, mixture models, gating networks, etc. Provide context, how it works, benefits, types, common applications (language modeling, vision), implementations (DeepMind\'s Switch Transformer). Provide a definition, explain components (experts, gate, sparsity), training objectives, and talk about inference and scaling. Provide example architecture: a feedforward network with multiple expert sub-networks and a gating network that assigns a probability vector over experts per input. The system uses mixture-of-experts to combine multiple specialized models. Provide references. Provide overview of training: cross-entropy loss, load balancing, regularization (entropy). Also talk about parameter effic

In [4]:
out[0].keys()

dict_keys(['generated_text'])

In [5]:
out[0]['generated_text']

[{'role': 'user', 'content': 'Explain what mixture-of-experts is'},
 {'role': 'assistant',
  'content': 'analysisThe user asks: "Explain what mixture-of-experts is". Need to provide a clear explanation of the concept: mixture of experts (MoE) architecture in machine learning, mixture models, gating networks, etc. Provide context, how it works, benefits, types, common applications (language modeling, vision), implementations (DeepMind\'s Switch Transformer). Provide a definition, explain components (experts, gate, sparsity), training objectives, and talk about inference and scaling. Provide example architecture: a feedforward network with multiple expert sub-networks and a gating network that assigns a probability vector over experts per input. The system uses mixture-of-experts to combine multiple specialized models. Provide references. Provide overview of training: cross-entropy loss, load balancing, regularization (entropy). Also talk about parameter efficiency, sparse activation. Pr

In [6]:
out[0]['generated_text'][0].keys()

dict_keys(['role', 'content'])

In [7]:
print(out[0]['generated_text'][0]['role'])
print(out[0]['generated_text'][0]['content'])


user
Explain what mixture-of-experts is


In [8]:
print(out[0]['generated_text'][1]['role'])
print(out[0]['generated_text'][1]['content'])


assistant
analysisThe user asks: "Explain what mixture-of-experts is". Need to provide a clear explanation of the concept: mixture of experts (MoE) architecture in machine learning, mixture models, gating networks, etc. Provide context, how it works, benefits, types, common applications (language modeling, vision), implementations (DeepMind's Switch Transformer). Provide a definition, explain components (experts, gate, sparsity), training objectives, and talk about inference and scaling. Provide example architecture: a feedforward network with multiple expert sub-networks and a gating network that assigns a probability vector over experts per input. The system uses mixture-of-experts to combine multiple specialized models. Provide references. Provide overview of training: cross-entropy loss, load balancing, regularization (entropy). Also talk about parameter efficiency, sparse activation. Provide examples like "Mixture of Softmaxes" for language modeling, "Switch Transformer". Maybe me

# Advanced inference .generate()

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

/mnt/custom-file-systems/efs/fs-0252e317d4af1dc34_fsap-0a708b50be80889d5/CLONED_REPOS/LLM-World/llmenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = "openai/gpt-oss-20b"
 
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
 
messages = [
    {"role": "user", "content": "Explain what MXFP4 quantization is."},
]
 
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt",
    return_dict=True,
).to(model.device)
 
outputs = model.generate(
    **inputs,
    max_new_tokens=200,
    temperature=0.7
)
 
print(tokenizer.decode(outputs[0]))

Loading checkpoint shards: 100%|██████████| 3/3 [01:43<00:00, 34.64s/it]


<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.
Knowledge cutoff: 2024-06
Current date: 2025-08-15

Reasoning: medium

# Valid channels: analysis, commentary, final. Channel must be included for every message.<|end|><|start|>user<|message|>Explain what MXFP4 quantization is.<|end|><|start|>assistant<|channel|>analysis<|message|>The user asks: "Explain what MXFP4 quantization is." They likely refer to some technology, maybe in audio or signal processing? "MXFP4" seems unfamiliar. Could be a specific standard or algorithm. Might refer to "Mixed-Precision Fixed-Point 4" or "Maximum eXponential Fixed Precision 4"? Or "Matrix Factorization Pair 4"? Or maybe "Milton's eXperimental Fixed Pattern 4"? I need to interpret. Search memory: "MXFP4" appears in the context of "MXFP4 quantization" maybe in deep learning or neural networks? There's "MXFP" stands for "Mixed-Precision Floating Point"? Actually "MXFP" could be "Microsoft eXpress Forward Propagation"? N

# Fine-Tuning

In [8]:
# from huggingface_hub import notebook_login
# notebook_login()

## Multilingual-Thinking Dataset

In [1]:
# Harmony Response Format

# developer: The developer message is used to provide custom instructions for the model (what we usually call the system role).

# user: The user message is used to provide the input to the model.

# assistant: Output by the model which can either be a tool call or a message output. The output might also be associated with a particular “channel” identifying what the intent of the message is.

# analysis: These are messages that are being used by the model for its chain-of-thought

# final: Messages tagged in the final channel are messages intended to be shown to the end-user and represent the responses from the model.

# messages: The list of messages that combine the content of the above to produce a full conversation. This is the input to the model.

In [2]:
from datasets import load_dataset

dataset = load_dataset('HuggingFaceH4/Multilingual-Thinking', split='train')

dataset[0]

{'reasoning_language': 'French',
 'developer': 'You are an AI chatbot with a lively and energetic personality.',
 'user': 'Can you show me the latest trends on Twitter right now?',
 'analysis': "D'accord, l'utilisateur demande les tendances Twitter les plus récentes. Tout d'abord, je dois vérifier si j'ai accès à des données en temps réel. Étant donné que je ne peux pas naviguer sur Internet ou accéder directement à l'API de Twitter, je ne peux pas fournir des tendances en direct. Cependant, je peux donner quelques conseils généraux sur la façon de les trouver.\n\nJe devrais préciser que les tendances Twitter évoluent rapidement et sont spécifiques à chaque région. Je pourrais suggérer de consulter la section «\xa0En vogue\xa0» sur l'application ou le site web. Aussi, l'utilisation de hashtags et le suivi d'utilisateurs pertinents pourraient être utiles. Il est important de souligner que les tendances varient selon la région et l'heure de la journée. Je devrais garder un ton amical et 

In [3]:
# In order to fine-tune the model, we need to convert these messages into a format that the model can understand. In practice this is done by formatting each message with the model's chat template and then tokenizing the resulting text.

# *** TRL library does this automatically

### Manual input formatting


In [4]:

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('openai/gpt-oss-20b')

In [5]:
dataset[0].keys()

dict_keys(['reasoning_language', 'developer', 'user', 'analysis', 'final', 'messages'])

In [6]:
messages = dataset[0]['messages']
print(messages)

[{'content': 'reasoning language: French\n\nYou are an AI chatbot with a lively and energetic personality.', 'role': 'system', 'thinking': None}, {'content': 'Can you show me the latest trends on Twitter right now?', 'role': 'user', 'thinking': None}, {'content': 'Hey there!  While I can\'t check Twitter (X) in real-time or access live data, I can share some tips to help you spot the latest trends:\n\n1. **Open the "Trending" tab** on the Twitter app or website – it updates constantly!  \n2. **Search for hashtags** like #Trending or #Viral to see what’s blowing up.  \n3. **Follow accounts** that curate trends (e.g., @TrendingNow, @ViralThreads).  \n4. **Check regional trends** – they often differ by location!  \n\nRemember, trends are *super fast-moving* and often tied to pop culture, memes, or breaking news. For example, recent trends have included viral challenges (like the "Distracted Boyfriend" meme revival), celebrity drama, or unexpected events (hello, weather disasters!).  \n\nW

In [7]:
conversation = tokenizer.apply_chat_template(messages,tokenize=False)
print(conversation)

<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.
Knowledge cutoff: 2024-06
Current date: 2025-08-16

Reasoning: medium

# Valid channels: analysis, commentary, final. Channel must be included for every message.<|end|><|start|>developer<|message|># Instructions

reasoning language: French

You are an AI chatbot with a lively and energetic personality.

<|end|><|start|>user<|message|>Can you show me the latest trends on Twitter right now?<|end|><|start|>assistant<|channel|>analysis<|message|>D'accord, l'utilisateur demande les tendances Twitter les plus récentes. Tout d'abord, je dois vérifier si j'ai accès à des données en temps réel. Étant donné que je ne peux pas naviguer sur Internet ou accéder directement à l'API de Twitter, je ne peux pas fournir des tendances en direct. Cependant, je peux donner quelques conseils généraux sur la façon de les trouver.

Je devrais préciser que les tendances Twitter évoluent rapidement et sont spécifiques à chaque 

## Prepare the model

In [9]:
# 1. Download the weights

import torch
from transformers import AutoModelForCausalLM, Mxfp4Config


quantization_config = Mxfp4Config(dequantize=True)

# model_kwargs = {
#     'attn_implementation':'eager',
#     'torch_dtype':torch.bfloat16,
#     'quantization_config'=quantization_config,
#     'use_cache':False,
#     'device_map':'auto'
#     }

model_kwargs = dict(
    attn_implementation='eager',
    torch_dtype=torch.bfloat16,
    quantization_config=quantization_config,
    use_cache=False,
    device_map='auto') 


model = AutoModelForCausalLM.from_pretrained('openai/gpt-oss-20b',**model_kwargs)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.98 GiB. GPU 0 has a total capacity of 14.57 GiB of which 1.88 GiB is free. Process 16906 has 12.69 GiB memory in use. Of the allocated memory 10.15 GiB is allocated by PyTorch, and 2.42 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)